# Imports

In [1]:
import pandas as pd

In [2]:
import nltk
import re

from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading vader_lexicon: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt_tab: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
import numpy as np
# def dummy_npwarn_decorator_factory():
#   def npwarn_decorator(x):
#     return x
#   return npwarn_decorator
# np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [4]:
from bertopic import BERTopic

d:\workdir\TopicModelling\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import pipeline #if error change version of transformers

# Dataset import and text preprocessing

In [6]:
df = pd.read_csv(r"D:\workdir\TopicModelling\Data\Car_Reviews_Database_KaggleSentiment.csv", encoding='ISO-8859-1')
df = df.dropna()
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 234 entries, 0 to 277
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    234 non-null    int64 
 1   Model   234 non-null    object
 2   Review  234 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


In [7]:
stop_words = set(stopwords.words('english'))
add_to_stop = ["car", "buick", "cadillac", "chevrolet", "chrysler", "dodge", "ford", "gmc", "honda", "hyundai", "jeep", "kia", "mazda", \
               "nissan", "subaru", "toyota", "volkswagen", "volvo"]
for word in add_to_stop:
    stop_words.add(word)
print(stop_words)

def preprocess_text(text: str, stopwords=stop_words) -> str:
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text) # leave only abc characters and spaces
    
    # remove stopwords
  
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if w.lower() not in stopwords]
    text = " ".join(tokens)
    text = text.lower().strip() # removes leading and trailing whitespaces
    
    return text

{'nissan', 'my', 'again', 'dodge', 'm', 'aren', 'not', "you'll", 'did', 'car', 'have', 'when', 'are', "should've", 'buick', 'yourselves', 'while', 'up', 'then', 'am', 'but', 'off', 'an', "haven't", 'from', 'same', 'on', 'me', 'we', 'was', "doesn't", 'them', "wouldn't", 'weren', 'further', 'of', "that'll", "mightn't", 'its', 'being', 'down', 'ourselves', 'both', 're', 'for', 'very', 'mustn', 've', 'where', 'because', 'why', 'just', 'all', 'y', "you're", 'here', "didn't", "mustn't", 'toyota', "don't", 'were', 'i', 'ain', 'kia', 'than', 'whom', 'needn', 'it', 'so', "shan't", 'he', 'chevrolet', 'mightn', 'shan', 'doesn', 'myself', 'each', 'or', 'their', 'd', 'in', 'has', 'if', "couldn't", "hasn't", 'during', 'couldn', 'to', 'with', 'she', 'wasn', 'cadillac', 'out', 'this', "you've", 'will', 's', 'these', 'hers', "won't", 'volvo', 'does', 'no', "wasn't", 'nor', 'only', 'such', 'didn', 'honda', "needn't", 'his', 'chrysler', "aren't", 'mazda', 'some', 'don', 'a', 'yourself', 'and', 'too', 'at

In [8]:
df['Review_processed'] = df['Review'].apply(lambda text: preprocess_text(text))
docs = df['Review_processed'].tolist()
df.sample(2)

,Year,Model,Review,Review_processed
30,2013,Hyundai,Hyundai Builds a Warm Hatch. While the Elantra...,builds warm hatch elantra sedan good compact p...
59,2017,GMC,"2017 GMC Acadia : All in all, GMC has met its ...",acadia met target creating smaller nimble refi...


# Experiments

## BERTopic

### Basics

In [9]:
topic_model = BERTopic()


In [10]:
topics, probs = topic_model.fit_transform(docs)

In [11]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,44,-1_model_also_luxurious_escape,"[model, also, luxurious, escape, world, mulsan...",[recently seen super luxurious model coming br...
1,0,144,0_drive_driving_good_like,"[drive, driving, good, like, great, even, well...",[vehicle long stood excess opulence knew going...
2,1,26,1_handling_engine_reliability_interior,"[handling, engine, reliability, interior, rati...",[despite athletic handling upscale cabin regal...
3,2,20,2_bmw_series_litres_rear,"[bmw, series, litres, rear, handling, luxury, ...",[bmw series muscular engines nimble handling a...


In [12]:
topic_model.get_topic(0)

[('drive', np.float64(0.029194358235344393)),
 ('driving', np.float64(0.0284785358946744)),
 ('good', np.float64(0.027089537218647392)),
 ('like', np.float64(0.026985884127588275)),
 ('great', np.float64(0.0252137963900265)),
 ('even', np.float64(0.023157299754122285)),
 ('well', np.float64(0.02260747604552674)),
 ('speed', np.float64(0.02206257388595158)),
 ('engine', np.float64(0.02121818970725095)),
 ('one', np.float64(0.020598780878446805))]

In [13]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,although arguably first generation insight hyb...,-1,-1_model_also_luxurious_escape,"[model, also, luxurious, escape, world, mulsan...",[recently seen super luxurious model coming br...,model - also - luxurious - escape - world - mu...,0.000000,False
1,accord ex l comfortable sporty cylinders best ...,0,0_drive_driving_good_like,"[drive, driving, good, like, great, even, well...",[vehicle long stood excess opulence knew going...,drive - driving - good - like - great - even -...,0.809251,False
2,owed driven products years purchased vehicle m...,0,0_drive_driving_good_like,"[drive, driving, good, like, great, even, well...",[vehicle long stood excess opulence knew going...,drive - driving - good - like - great - even -...,0.878965,False
3,accord euro l seats average little rear legroo...,-1,-1_model_also_luxurious_escape,"[model, also, luxurious, escape, world, mulsan...",[recently seen super luxurious model coming br...,model - also - luxurious - escape - world - mu...,0.000000,False
4,hr v continuous variable transmission failed r...,0,0_drive_driving_good_like,"[drive, driving, good, like, great, even, well...",[vehicle long stood excess opulence knew going...,drive - driving - good - like - great - even -...,1.000000,False
...,...,...,...,...,...,...,...,...
229,civic ex clever design spacious really good sp...,0,0_drive_driving_good_like,"[drive, driving, good, like, great, even, well...",[vehicle long stood excess opulence knew going...,drive - driving - good - like - great - even -...,0.969334,False
230,sonata large spacious plenty front rear leg ro...,0,0_drive_driving_good_like,"[drive, driving, good, like, great, even, well...",[vehicle long stood excess opulence knew going...,drive - driving - good - like - great - even -...,0.906765,False
231,deal prefer rear wheel drive amazing vehicle f...,0,0_drive_driving_good_like,"[drive, driving, good, like, great, even, well...",[vehicle long stood excess opulence knew going...,drive - driving - good - like - great - even -...,0.965994,False
232,escape xlt really want like purchased necessit...,-1,-1_model_also_luxurious_escape,"[model, also, luxurious, escape, world, mulsan...",[recently seen super luxurious model coming br...,model - also - luxurious - escape - world - mu...,0.000000,False


### Fine-tune Topic Representations

In [14]:
from bertopic.representation import KeyBERTInspired

In [15]:
representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model)

In [16]:
topics, probs = topic_model.fit_transform(docs)

In [17]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,38,-1_luxurious_mercedes_bmw_richest,"[luxurious, mercedes, bmw, richest, bentley, c...",[recently seen super luxurious model coming br...
1,0,148,0_audi_vehicle_cars_driving,"[audi, vehicle, cars, driving, suv, highway, e...",[vehicle long stood excess opulence knew going...
2,1,28,1_automotive_regal_seats_luxurious,"[automotive, regal, seats, luxurious, sedan, c...",[despite athletic handling upscale cabin regal...
3,2,20,2_bmw_mercedes_sedan_luxury,"[bmw, mercedes, sedan, luxury, audi, luxurious...",[bmw series muscular engines nimble handling a...
